## Solving overfitting

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 100

In [3]:
# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv3_1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## Prepare cifar10 dataset

In [4]:
cifar10 = tf.keras.datasets.cifar10 # 32x32x3 -> 10 classes

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

170500096/170498071 [==============================] - 22s 0us/step


## Keras API model training

In [12]:
model = MyModel()
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 1563 steps, validate for 313 steps
Epoch 1/100
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4330 - accuracy: 0.4752 - val_loss: 1.1082 - val_accuracy: 0.6087
Epoch 2/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.9477 - accuracy: 0.6654 - val_loss: 0.8525 - val_accuracy: 0.7033
Epoch 3/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7330 - accuracy: 0.7406 - val_loss: 0.7960 - val_accuracy: 0.7236
Epoch 4/100
1563/1563 [==============================] - 5s 4ms/step - loss: 0.5612 - accuracy: 0.8037 - val_loss: 0.8482 - val_accuracy: 0.7338
Epoch 5/100
1563/1563 [==============================] - 5s 4ms/step - loss: 0.3933 - accuracy: 0.8627 - val_loss: 0.8878 - val_accuracy: 0.7297
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2456 - accuracy: 0.9142 - val_loss: 1.1325 - val_accuracy: 0.7319
Epoch 7/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1713 -

Epoch 57/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0760 - accuracy: 0.9842 - val_loss: 3.8667 - val_accuracy: 0.7049
Epoch 58/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0686 - accuracy: 0.9850 - val_loss: 4.2643 - val_accuracy: 0.6942
Epoch 59/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0823 - accuracy: 0.9828 - val_loss: 4.1043 - val_accuracy: 0.7021
Epoch 60/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0732 - accuracy: 0.9849 - val_loss: 3.8420 - val_accuracy: 0.7079
Epoch 61/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0709 - accuracy: 0.9846 - val_loss: 4.0454 - val_accuracy: 0.6918
Epoch 62/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0763 - accuracy: 0.9842 - val_loss: 4.4040 - val_accuracy: 0.7020
Epoch 63/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0713 - accuracy: 0.9851 - val_loss: 4.1435 -

## Improve architecture

In [10]:
class ConvLNReluBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvLNReluBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
        self.ln = tf.keras.layers.LayerNormalization()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        x = self.ln(x) # LayerNormalization does not affect training weights
        return tf.nn.relu(x)
        

# Define network architecture
class MyModelLN(tf.keras.Model):
    def __init__(self):
        super(MyModelLN, self).__init__()
        self.conv1_1 = ConvLNReluBlock(16, (3, 3))
        self.conv1_2 = ConvLNReluBlock(16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv2_1 = ConvLNReluBlock(32, (3, 3))
        self.conv2_2 = ConvLNReluBlock(32, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv3_1 = ConvLNReluBlock(64, (3, 3))
        self.conv3_2 = ConvLNReluBlock(64, (3, 3))
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu', 
                                            kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax', 
                                            kernel_regularizer = tf.keras.regularizers.l2(0.01))
        
    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [13]:
model = MyModelLN()
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 1563 steps, validate for 313 steps
Epoch 1/100
1563/1563 [==============================] - 12s 8ms/step - loss: 2.2318 - accuracy: 0.3800 - val_loss: 1.4979 - val_accuracy: 0.5092
Epoch 2/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2661 - accuracy: 0.5907 - val_loss: 1.1265 - val_accuracy: 0.6449
Epoch 3/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.0893 - accuracy: 0.6579 - val_loss: 1.0022 - val_accuracy: 0.6851
Epoch 4/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9894 - accuracy: 0.6940 - val_loss: 1.0308 - val_accuracy: 0.6814
Epoch 5/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9212 - accuracy: 0.7185 - val_loss: 0.9180 - val_accuracy: 0.7172
Epoch 6/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8715 - accuracy: 0.7366 - val_loss: 0.8514 - val_accuracy: 0.7431
Epoch 7/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0

1563/1563 [==============================] - 11s 7ms/step - loss: 0.2810 - accuracy: 0.9483 - val_loss: 0.9687 - val_accuracy: 0.7774
Epoch 57/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2773 - accuracy: 0.9496 - val_loss: 0.9460 - val_accuracy: 0.7727
Epoch 58/100
1563/1563 [==============================] - 12s 7ms/step - loss: 0.2779 - accuracy: 0.9488 - val_loss: 1.0850 - val_accuracy: 0.7621
Epoch 59/100
1563/1563 [==============================] - 12s 7ms/step - loss: 0.2701 - accuracy: 0.9514 - val_loss: 1.0216 - val_accuracy: 0.7797
Epoch 60/100
1563/1563 [==============================] - 12s 7ms/step - loss: 0.2644 - accuracy: 0.9531 - val_loss: 1.0014 - val_accuracy: 0.7789
Epoch 61/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2666 - accuracy: 0.9529 - val_loss: 1.0192 - val_accuracy: 0.7766
Epoch 62/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2623 - accuracy: 0.9542 - val_loss: 1.0700 - val_a